In [ ]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models

pandas = pd.read_table("tripdata.dat")
database = db.Database("tripdata",pandas)

# The Pandas data structure is available as database.data. Use all the
# Pandas functions to invesigate the database
#print(database.data.describe())

from headers import *

Below are all of the variables declarations:

In [ ]:
# Alternative specific constants
ASC_AUTO_DRIVE = Beta('ASC_AUTO_DRIVE',0,None,None,0)
ASC_AUTO_PASS = Beta('ASC_AUTO_PASS',0,None,None,0)
ASC_METRO = Beta('ASC_METRO',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_WALK = Beta('ASC_WALK',0,None,None,1)

# Sociodemo variables
B_SEX = Beta('B_SEX',0,None,None,0) # 1 if female; 0 if otherwise
B_AGE = Beta('B_AGE',0,None,None,0) # age
B_EMP = Beta('B_EMP',0,None,None,0) # parttime or fulltime
B_WKWK = Beta('B_WKWK',0,None,None,0) # work week
B_INC = Beta('B_INC',0,None,None,0) # income
B_HH = Beta('B_HH',0,None,None,0) # household
B_NVEH = Beta('B_NVEH',0,None,None,0) # number of household vehicles

# Generalized travel variables
B_IVTT = Beta('B_IVTT',0,None,None,0) # in vehicle travel time
B_DIST = Beta('B_DIST',0,None,None,0) # distance

# Car-related variables
B_DLIC = Beta('B_DLIC',0,None,None,0) # drivers license 1 or 0

# Train-related variables
B_NTERM = Beta('B_NTERM',0,None,None,0) # commuter train access station
B_GFARE = Beta('B_GFARE',0,None,100,0) # commuter train fare
B_GWAIT = Beta('B_GWAIT',0,None,100,0) # commuter train wait time

# Metro-related variables
B_MSUB = Beta('B_MSUB',0,None,None,0) # closest metro station to destination
B_MDIS = Beta('B_MDIS',0,None,None,0) # distance from metro to destination
B_TFARE = Beta('B_TFARE',0,None,None,0) # transit fare
B_TWAIT = Beta('B_TWAIT',0,None,None,0) # transit wait time

MU = Beta('MU',2.05,1,10,0)
MU_2 = Beta('MU_2',2.05,1,10,0)

The next cell defines new variables in order to exclude crazy values

In [ ]:
# Updating variables to account for 'missing' values 
nveh_fix =  DefineVariable('nveh_fix', nveh * (nveh != 9), database)
gfare_fix = DefineVariable('gfare_fix', gfare * (gfare != 9999), database)
tfare_fix = DefineVariable('tfare_fix', tfare * (tfare != 9999), database)
gwait_fix = DefineVariable('gwait_fix', gwait * (gwait != 9999), database)
twait_fix = DefineVariable('twait_fix', twait * (twait != 9999), database)

In [ ]:
V1 = ASC_AUTO_DRIVE + \
     B_DIST * trdist + \
     B_DLIC * dlic + \
     B_IVTT * ivtt1
V2 = ASC_AUTO_PASS + \
     B_IVTT * ivtt2
V3 = ASC_METRO + \
     B_TFARE * tfare_fix + \
     B_IVTT * ivtt3
V4 = ASC_TRAIN + \
     B_DIST * trdist + \
     B_GFARE * gfare_fix + \
     B_NTERM * nterm + \
     B_IVTT * ivtt4
V5 = ASC_WALK + \
     B_SEX * female + \
     B_IVTT * ivtt5

In [ ]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5}

# Associate the availability conditions with the alternatives

av = {1: avail1,
      2: avail2,
      3: avail3,
      4: avail4,
      5: avail5}

#Definition of nests:
# 1: nests parameter
# 2: list of alternatives
solo = 1.0 , [1,5]
shared = MU, [2,3,4]
nests = solo,shared

logprob = models.lognested(V,av,nests,choice)
biogeme  = bio.BIOGEME(database,logprob) # This is my log likelihood function
biogeme.modelName = "Nested_Model"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)